In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
raw = 'raw'
processed = 'processed'
cache = 'cache'

In [3]:
!mkdir -p {processed}

In [ ]:
!mkdir -p {cache}

In [17]:
import pandas as pd
from IPython.display import display
def no_display(val):
    pass

In [26]:
%%time
def sales_train_melt(display = no_display):
    df_sales_train = pd.read_csv(f"{raw}/sales_train_validation.csv", nrows=1)

    id_columns = [col for col in df_sales_train.columns if 'id' in col]
    display(id_columns)

    sales_columns = [col for col in df_sales_train.columns if 'd_' in col]
    display([sales_columns[0], sales_columns[-1]])

    cat_columns = [col for col in id_columns if col != 'id']
    display(cat_columns)

    """
    # No dtype: <class 'pandas.core.frame.DataFrame'>
    RangeIndex: 30490 entries, 0 to 30489
    Columns: 1919 entries, id to d_1913
    dtypes: int64(1913), object(6)
    memory usage: 446.4+ MB
    CPU times: user 5.77 s, sys: 677 ms, total: 6.44 s
    Wall time: 6.61 s

    # With dtype:
    <class 'pandas.core.frame.DataFrame'>
    RangeIndex: 30490 entries, 0 to 30489
    Columns: 1919 entries, id to d_1913
    dtypes: category(6), int64(1913)
    memory usage: 446.8 MB
    CPU times: user 5.42 s, sys: 507 ms, total: 5.93 s
    Wall time: 5.96 s"""
    df_sales_train = pd.read_csv(f"{raw}/sales_train_validation.csv")
    display(df_sales_train.info())

    """
    # No dtype on input:
    CPU times: user 9.88 s, sys: 2.07 s, total: 11.9 s
    Wall time: 12 s

    # dtype on input (even after restarting the kernel :/)
    CPU times: user 2min 32s, sys: 8.3 s, total: 2min 40s
    Wall time: 2min 42s

    Note to self: applying categories on top of un-dtyped is mucho fastor overalle

    """
    df_sales_train_melt = df_sales_train.melt(
        id_vars=id_columns,
        var_name='day_id',
        value_name='sales'
    )

    """
    for col in id_columns:
        df_sales_train_melt[col] = df_sales_train_melt[col].astype('category')
    CPU times: user 28.7 s, sys: 6.33 s, total: 35 s
    Wall time: 35.3 s
    """
    for col in id_columns:
        df_sales_train_melt[col] = df_sales_train_melt[col].astype('category')

    """
    # No dtype on input: 
    <class 'pandas.core.frame.DataFrame'>
    RangeIndex: 58327370 entries, 0 to 58327369
    Data columns (total 8 columns):
    id          object
    item_id     object
    dept_id     object
    cat_id      object
    store_id    object
    state_id    object
    day_id      object
    sales       int64
    dtypes: int64(1), object(7)
    memory usage: 3.5+ GB
    """
    display(df_sales_train_melt.info())

    # Note to self: dropping derivable id strings doesn't seem to buy much mem
    """<class 'pandas.core.frame.DataFrame'>
    RangeIndex: 58327370 entries, 0 to 58327369
    Data columns (total 7 columns):
    item_id     category
    dept_id     category
    cat_id      category
    store_id    category
    state_id    category
    day_id      object
    sales       int64
    dtypes: category(5), int64(1), object(1)
    memory usage: 1.2+ GB                    (vs 1.3G)
    display(df_sales_train_melt.drop('id', axis=1).info())
    """
    return df_sales_train_melt

df_sales_train_melt = sales_train_melt(display=display)

['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

['d_1', 'd_1913']

['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 8 columns):
id          category
item_id     category
dept_id     category
cat_id      category
store_id    category
state_id    category
day_id      object
sales       int64
dtypes: category(6), int64(1), object(1)
memory usage: 1.3+ GB


None

CPU times: user 46.4 s, sys: 10.3 s, total: 56.7 s
Wall time: 58.2 s


In [28]:
%%time
"""
CPU times: user 3min 55s, sys: 11.1 s, total: 4min 6s
Wall time: 4min 18s

Note to self: saving melt stuff back is more expensive than reading / processing it from scratch o_O """
df_sales_train_melt.to_csv(f'{processed}/sales_train_melt.csv', index=False)

CPU times: user 3min 55s, sys: 11.1 s, total: 4min 6s
Wall time: 4min 18s


In [29]:
!ls processed

sales_train_melt.csv
